# This is the Coursera capstone project (Seattle collisions)

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
#https://data-seattlecitygis.opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0
dfOriginal = pd.read_csv('Data-Collisions.csv')

C:\Users\carniato\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data cleaning

Inspect data first

In [4]:
def computeDetails(df): 
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()])
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    missing_ration = (df.isnull().sum()/ obs) * 100
    
    cols = ['types', 'counts', 'distincts', 'nulls', 'missing ration']
    details = pd.concat([types, counts, distincts, nulls, missing_ration], axis = 1)
    details.columns = cols
    
    return details

def GetIndexses(df,col_missing_data,missingValue):
    if not missingValue :
        # nan
        indexses = df[col_missing_data].isnull()
    else:
        # comparables
        indexses = df[col_missing_data]==missingValue
        
    return indexses

def InputMissingData(df, col_missing_data, col_ref, missingValue = None):
    #group by nan or missing value
    invalid_idxs = GetIndexses(df, col_missing_data, missingValue)
    df_notval = df[invalid_idxs]
    df_notval = df_notval.groupby(col_ref).sum()  
    #print(df_notval[col_ref])
    #the valid entries
    df_val = df[~invalid_idxs]
    
    #now check the values of the reference column
    for r in df_notval.index.values:
        if isinstance(r, str):
            #a string (e.g. a location)
            df_loc = df_val[df_val[col_ref]==r]
        elif len(r)==2:
            #two doubles
            df_loc = df_val[(df_val[col_ref[0]]==r[0]) & (df_val[col_ref[1]]==r[1])]
        
        if len(df_loc) > 0:
            val = df_loc[col_missing_data].iloc[0]
            if isinstance(r, str):
                #substitute with the first valid value
                df.loc[(df[col_ref]==r) & invalid_idxs,col_missing_data]=val
            elif len(r) ==2:
                df.loc[(df[col_ref[0]]==r[0]) &(df[col_ref[1]]==r[1]) & invalid_idxs, col_missing_data] = val
        
    return df        

In [5]:
df = dfOriginal.copy()
details = computeDetails(df)
print(details)

                  types  counts  distincts   nulls  missing ration
SEVERITYCODE      int64  194673          2       0           0.000
X               float64  189339      23564    5334           2.740
Y               float64  189339      23840    5334           2.740
OBJECTID          int64  194673     194673       0           0.000
INCKEY            int64  194673     194673       0           0.000
COLDETKEY         int64  194673     194673       0           0.000
REPORTNO         object  194673     194670       0           0.000
STATUS           object  194673          2       0           0.000
ADDRTYPE         object  192747          4    1926           0.989
INTKEY          float64   65070       7615  129603          66.575
LOCATION         object  191996      24103    2677           1.375
EXCEPTRSNCODE    object   84811          3  109862          56.434
EXCEPTRSNDESC    object    5638          2  189035          97.104
SEVERITYCODE.1    int64  194673          2       0           0

Eliminate the columns that are semantically equal or similar to the target

In [6]:
df = df.drop(['SEVERITYCODE.1'], axis=1)
df = df.drop(['SEVERITYDESC'], axis=1)

Eliminate EXCEPTRSNDESC, not enough information

In [7]:
df['EXCEPTRSNDESC'].value_counts()

Not Enough Information, or Insufficient Location Information    5638
Name: EXCEPTRSNDESC, dtype: int64

In [8]:
df = df.drop(['EXCEPTRSNDESC'], axis=1)

EXCEPTRSNCODE has no descriptive field 

In [9]:
df = df.drop(['EXCEPTRSNCODE'], axis=1)

INATTENTIONIND id contain only yes, suppose the other equal to n

In [10]:
df['INATTENTIONIND'].value_counts()

Y    29805
Name: INATTENTIONIND, dtype: int64

In [11]:
df['INATTENTIONIND'] = df['INATTENTIONIND'].replace(np.nan,'N')
df['INATTENTIONIND'].value_counts()

N    164868
Y     29805
Name: INATTENTIONIND, dtype: int64

Drop initkey

In [14]:
df = df.drop(['INTKEY'], axis=1)

In [15]:
details = computeDetails(df)
print(details)

                  types  counts  distincts   nulls  missing ration
SEVERITYCODE      int64  194673          2       0           0.000
X               float64  189339      23564    5334           2.740
Y               float64  189339      23840    5334           2.740
OBJECTID          int64  194673     194673       0           0.000
INCKEY            int64  194673     194673       0           0.000
COLDETKEY         int64  194673     194673       0           0.000
REPORTNO         object  194673     194670       0           0.000
STATUS           object  194673          2       0           0.000
ADDRTYPE         object  192747          4    1926           0.989
LOCATION         object  191996      24103    2677           1.375
COLLISIONTYPE    object  189769         11    4904           2.519
PERSONCOUNT       int64  194673         47       0           0.000
PEDCOUNT          int64  194673          7       0           0.000
PEDCYLCOUNT       int64  194673          3       0           0

In [18]:
df['PEDROWNOTGRNT'].value_counts()

Y    4667
Name: PEDROWNOTGRNT, dtype: int64

In [19]:
df['PEDROWNOTGRNT'] = df['PEDROWNOTGRNT'].replace(np.nan,'N')
df['PEDROWNOTGRNT'].value_counts()

N    190006
Y      4667
Name: PEDROWNOTGRNT, dtype: int64

In [21]:
df['SPEEDING'] = df['SPEEDING'].replace(np.nan,'N')
df['SPEEDING'].value_counts()

N    185340
Y      9333
Name: SPEEDING, dtype: int64

In [23]:
details = computeDetails(df)
print(details)

                  types  counts  distincts  nulls  missing ration
SEVERITYCODE      int64  194673          2      0           0.000
X               float64  189339      23564   5334           2.740
Y               float64  189339      23840   5334           2.740
OBJECTID          int64  194673     194673      0           0.000
INCKEY            int64  194673     194673      0           0.000
COLDETKEY         int64  194673     194673      0           0.000
REPORTNO         object  194673     194670      0           0.000
STATUS           object  194673          2      0           0.000
ADDRTYPE         object  192747          4   1926           0.989
LOCATION         object  191996      24103   2677           1.375
COLLISIONTYPE    object  189769         11   4904           2.519
PERSONCOUNT       int64  194673         47      0           0.000
PEDCOUNT          int64  194673          7      0           0.000
PEDCYLCOUNT       int64  194673          3      0           0.000
VEHCOUNT  

we could try to fill the missing 'JUNCTIONTYPE' by looking X,Y?

In [24]:
original_length = len(df['JUNCTIONTYPE'])
df = InputMissingData(df, 'JUNCTIONTYPE', ['X','Y'])
df_dropped = df.dropna()
print("percentage dropped ", (original_length-len(df_dropped))/len(df))

percentage dropped  0.9998972636164234


In [22]:
df.loc[df['INTKEY']==33512.000, 'JUNCTIONTYPE'].value_counts()

At Intersection (intersection related)    128
Name: JUNCTIONTYPE, dtype: int64

In [17]:
#JUNCTIONTYPE
df['JUNCTIONTYPE'].value_counts()

Mid-Block (not related to intersection)              89800
At Intersection (intersection related)               62810
Mid-Block (but intersection related)                 22790
Driveway Junction                                    10671
At Intersection (but not related to intersection)     2098
Ramp Junction                                          166
Unknown                                                  9
Name: JUNCTIONTYPE, dtype: int64

In [ ]:
df = df.drop(['OBJECTID','INCKEY','COLDETKEY','REPORTNO','ST_COLCODE','ST_COLDESC','SDOTCOLNUM'], axis=1)
df = df.drop(['COLLISIONTYPE','SDOT_COLCODE','SDOT_COLDESC'], axis=1)

Remove columns with a ratio of missing values > 50%

In [ ]:
df = dfOriginal.drop(['INTKEY','EXCEPTRSNCODE','EXCEPTRSNDESC','INATTENTIONIND','PEDROWNOTGRNT','SPEEDING'], axis=1)

In [ ]:
details = computeDetails(df)
print(details)

# Eliminate the columns that are semantically equal or similar to the target

Some fatures are administrative records, after the incided occoured

In [ ]:
df = df.drop(['OBJECTID','INCKEY','COLDETKEY','REPORTNO','ST_COLCODE','ST_COLDESC','SDOTCOLNUM'], axis=1)
df = df.drop(['COLLISIONTYPE','SDOT_COLCODE','SDOT_COLDESC'], axis=1)
details = computeDetails(df)
print(details)

# Use Location for missing X,Y, Junction type

In [ ]:
original_length = len(df)
df_dropped = df.dropna()
print("percentage dropped ", (original_length-len(df_dropped))/len(df))

In [ ]:
df = InputMissingData(df, 'X', 'LOCATION')
df = InputMissingData(df, 'Y', 'LOCATION')


In [ ]:
df_dropped = df.dropna()
print("percentage dropped ", (original_length-len(df_dropped))/len(df))

# Treat the dates

In [ ]:
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])

In [ ]:
df = df.drop(['INCDATE'], axis=1)

check incidents by hour

In [ ]:
df['hour'] = df['INCDTTM'].dt.hour
df_selected = df[df['SEVERITYCODE']==1]
sns.distplot(df_selected['hour'],kde = False)
df_selected = df[df['SEVERITYCODE']==2]
sns.distplot(df_selected['hour'],kde = False)

looks like there are way too many 0 hours, hour cannot be used as feature

In [ ]:
#df = df.drop(['hour'], axis=1)

Check day of the week

In [ ]:
df['dayofweek'] = df['INCDTTM'].dt.dayofweek
df_selected = df[df['SEVERITYCODE']==1]
sns.distplot(df_selected['dayofweek'],kde = False)
df_selected = df[df['SEVERITYCODE']==2]
sns.distplot(df_selected['dayofweek'],kde = False)

Check month

In [ ]:
df['month'] = df['INCDTTM'].dt.month
df_selected = df[df['SEVERITYCODE']==1]
sns.distplot(df_selected['month'],kde = False)
df_selected = df[df['SEVERITYCODE']==2]
sns.distplot(df_selected['month'],kde = False)

In [ ]:
df = df.drop(['INCDTTM'], axis=1)

# Use LOCATIONS instead of X and Y, remove STATUS (matched/unmached) 

In [ ]:
df = df.drop(['LOCATION','STATUS'], axis=1)

# UNDERINFL contains both Y/N and 0/1

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['UNDERINFL'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
def prepare_categorical(toLabel):
    le = LabelEncoder()
    le.fit(toLabel)
    toLabel_enc = le.transform(toLabel)
    return toLabel_enc

In [ ]:
toLabel_enc = prepare_categorical(df['UNDERINFL'])
df['UNDERINFL'] = toLabel_enc

In [ ]:
df['UNDERINFL'].value_counts()

In [ ]:
df['UNDERINFL'] = df['UNDERINFL'].replace(2,0)
df['UNDERINFL'] = df['UNDERINFL'].replace(3,1)       

In [ ]:
df['UNDERINFL'].value_counts()

In [ ]:
details = computeDetails(df)
print(details)

# Save cleaned up dataframe

In [ ]:
df.dropna(inplace=True)
df.to_csv('Data-Collisions-cleaned.csv')

# Load cleaned dataframe

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df = pd.read_csv('Data-Collisions-cleaned.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.dropna(inplace=True)
#details = computeDetails(df)
#print(details)
df.describe()

In [ ]:
#https://machinelearningmastery.com/feature-selection-with-categorical-data/
from sklearn.preprocessing import LabelEncoder
def prepare_categorical(toLabel):
    le = LabelEncoder()
    le.fit(toLabel)
    toLabel_enc = le.transform(toLabel)
    return toLabel_enc

df['ADDRTYPE'] = prepare_categorical(df['ADDRTYPE'])
#df['LOCATION'] = prepare_categorical(df['LOCATION'])
df['JUNCTIONTYPE'] = prepare_categorical(df['JUNCTIONTYPE'])
df['UNDERINFL'] = prepare_categorical(df['UNDERINFL'])
df['WEATHER'] = prepare_categorical(df['WEATHER'])
df['ROADCOND'] = prepare_categorical(df['ROADCOND'])
df['LIGHTCOND'] = prepare_categorical(df['LIGHTCOND'])
df['HITPARKEDCAR'] = prepare_categorical(df['HITPARKEDCAR'])


df = df.drop(['CROSSWALKKEY','SEGLANEKEY','PEDCOUNT',
              'HITPARKEDCAR','ADDRTYPE','JUNCTIONTYPE','WEATHER','ROADCOND','LIGHTCOND',
              'UNDERINFL','VEHCOUNT','dayofweek','hour','month','PERSONCOUNT' ,'X','Y'], axis=1)

#details = computeDetails(df)
#print(details)
#df = df.iloc[:30000,:]
df.describe()

# Benchmark a first tree model

In [ ]:
y = df['SEVERITYCODE']
X = df.drop(['SEVERITYCODE'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

train_scaler = StandardScaler()
X_train = train_scaler.fit_transform(X_train)
test_scaler = StandardScaler()
X_test = train_scaler.fit_transform(X_test)

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy", class_weight='balanced')
clf.fit(X_train, y_train) 
yhat = clf.predict(X_test)
f1_score(y_test, yhat, average='weighted') 

try imblearn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
rus = RandomOverSampler()
X_rus, y_rus = rus.fit_sample(X_train, y_train)
print('X_train len ', len(X_train),' len X_rus', len(X_rus))

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy")
clf.fit(X_rus, y_rus) 
yhat = clf.predict(X_test)
f1_score(y_test, yhat, average='weighted') 